In [19]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [20]:
data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')

data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

data_f_1=pd.read_excel('/root/GDUT/附表1-检索表格-流水号vs时间.xlsx')
data_f_time=pd.read_excel('时间点.xlsx')

In [21]:
# 取1a问题对应需要用的数据
# data1_columns = ['ID','数据集划分', '入院首次影像检查流水号', '发病到首次影像检查时间间隔']
data1_columns = ['ID','数据集划分', '入院首次影像检查流水号', '发病到首次影像检查时间间隔']
data1_1a = data1[data1_columns]

data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)
data2_columns = ['ID'] + [col for col in data2.columns if col.startswith('HM_volume') or col.startswith('随访')] # '首次检查流水号'
data2_1a = data2[data2_columns]

In [22]:
# 合并和保存数据
_1a=pd.merge(data1_1a,data2_1a,how='outer',on='ID')
_1a.to_excel('1a数据.xlsx', index=False)

In [23]:
# 转换为时间戳格式
columns_to_convert = data_f_time.columns[1:]
data_f_time[columns_to_convert] = data_f_time[columns_to_convert].apply(pd.to_datetime)
# Datetime对象转换为秒级的时间戳形式
data_f_time[columns_to_convert] = data_f_time[columns_to_convert].apply(lambda x: x.astype(int) // 10**9)
# 将负数的时间戳转变为空值
data_f_time[data_f_time.columns[1:]] = data_f_time[data_f_time.columns[1:]].apply(lambda x: x.mask(x < 0))

In [24]:
# 找到两个 DataFrame 共有的列名
common_columns = data_f_time.columns.intersection(_1a.columns)
# 使用 data_f_time 中的列替换 _1a 中的相同列名的列
_1a[common_columns] = data_f_time[common_columns]

In [25]:
# 取时间做散点图
result_list = []
flow_cols =[col for col in _1a.columns if col.endswith('流水号')]
hm_cols = [col for col in _1a.columns if col.startswith('HM_')]

# 修改寻访的变为以0开始的时间戳
cumulative_value_lists = []
p = 0 # 更新第几列
for i in range(8):
    cumulative_value_list = []
    q = 0
    for j,k,m in zip(_1a[flow_cols[i]], _1a[flow_cols[i+1]], _1a['发病到首次影像检查时间间隔']):
        if j == float(np.nan) or k == float(np.nan): # 无记录继承前面的记录
            cumulative_value = cumulative_value_lists[p-1][q]
        else:
            if i == 0:
                cumulative_value = 0
                cumulative_value += (m * 3600 + (k - j))
                cumulative_value_list.append(cumulative_value)
            else:
                cumulative_value = cumulative_value_lists[p-1][q] # 继承上一次检测的时间戳
                cumulative_value += (k - j)
                cumulative_value_list.append(cumulative_value)
        q += 1 # 更新第几行
    cumulative_value_lists.append(cumulative_value_list)
    p += 1

# 最后才更新数据，前面只存储了每个时间戳以0开始的值
for i in range(8):
    for j,k,m in zip(_1a[flow_cols[i]], _1a[flow_cols[i+1]], _1a['发病到首次影像检查时间间隔']):
        _1a[flow_cols[i+1]] = np.array(cumulative_value_lists[i])

# 修改首次入院检查时的时间戳
_1a['入院首次影像检查流水号'] = _1a['发病到首次影像检查时间间隔'] * 3600
# 此时_1a内流水号全为时间戳形式了

# 画总体的散点图
'''
# [(时间戳，体积大小)]
for i,j in zip(flow_cols, hm_cols):
    for l,m in zip(_1a[i],_1a[j]):
        if l <= 48 * 3600: # 只取48小时内的数据
            result_list.append((l,m))

# 去除空值再画图
result_list = [item for item in result_list if not any(math.isnan(value) for value in item)]

x_values = [item[0] for item in result_list]
y_values = [item[1] for item in result_list]

# 绘制散点图
plt.scatter(x_values, y_values)

# 添加标题和标签
plt.title('1a')
plt.xlabel('timestamp')
plt.ylabel('HM_volume')

# 显示图形
plt.show()
'''

selected_cols = []
for i,j in zip(flow_cols, hm_cols):
    selected_cols.append(i)
    selected_cols.append(j)
selected_df = _1a[selected_cols]
# selected_df['ID']=_1a['ID']

In [26]:
selected_df.to_excel('1a数据_已替换时间戳_已矫正.xlsx', index=False)

In [27]:
#线性
def liner_func(x,a,b):
    return a*x+b

#二次
def erchi_func(x,a,b,c):
    return a*x**2+b*x+c

#三次
def sanchi_func(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d

# 三角函数
def trig_func(x,a,b,c):
    return a*np.sin(x)+b*np.cos(x)+c
# 指数曲线
def target_func(x, a, b, c):
    return a * np.exp(-x / b) + c
# 对数函数
def hyp_func(x, a,b):
    return a*np.log(x)+b

def __sst(y_no_fitting):
    """
    计算SST(total sum of squares) 总平方和
    :param y_no_predicted: List[int] or array[int] 待拟合的y
    :return: 总平方和SST
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_no_fitting]
    sst = sum(s_list)
    return sst


def __ssr(y_fitting, y_no_fitting):
    """
    计算SSR(regression sum of squares) 回归平方和
    :param y_fitting: List[int] or array[int]  拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 回归平方和SSR
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_fitting]
    ssr = sum(s_list)
    return ssr


def __sse(y_fitting, y_no_fitting):
    """
    计算SSE(error sum of squares) 残差平方和
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 残差平方和SSE
    """
    s_list = [(y_fitting[i] - y_no_fitting[i])**2 for i in range(len(y_fitting))]
    sse = sum(s_list)
    return sse


def goodness_of_fit(y_fitting, y_no_fitting):
    """
    计算拟合优度R^2
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 拟合优度R^2
    """
    SSR = __ssr(y_fitting, y_no_fitting)
    SST = __sst(y_no_fitting)
    rr = SSR /SST
    return rr
def selected_func(you,model,model_select):
    maxyou=max(you)
    for s in range(len(you)):
        if you[s]==maxyou:
            
            return model[s],model_select[s]
        
    

def is_non_real(x):
    # 判断复数是否为非实数
    return math.isinf(x) or math.isnan(x)

def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1
        
#========================粒子群=======================================
import numpy as np



# def particle_swarm_optimization(mm, objective_func, num_particles, max_iterations,parm):
#     # 初始化参数
#     dimensions = 1
#     inertia = 0.5  # 惯性权重
#     cognitive_weight = 0.2  # 学习因子
#     social_weight = 0.2  # 学习因子
#     min_bound = 0  # 变量的最小边界
#     max_bound = 48  # 变量的最大边界

#     # 初始化粒子的位置和速度
#     particles = np.random.uniform(min_bound, max_bound, (num_particles, dimensions))
#     velocities = np.zeros((num_particles, dimensions))
#     if mm==1:
#         # 初始化粒子的局部最佳位置和全局最佳位置
#         personal_best_positions = particles.copy()
#         global_best_position = particles[np.argmin(objective_func(particles,parm[0],parm[1],parm[2],parm[3]))]

#         # 迭代更新粒子的速度和位置
#         for _ in range(max_iterations):
#             for i in range(num_particles):
#                 # 更新粒子的速度
#                 velocities[i] = (inertia * velocities[i] +
#                                 cognitive_weight * np.random.rand() * (personal_best_positions[i] - particles[i]) +
#                                 social_weight * np.random.rand() * (global_best_position - particles[i]))

#                 # 限制速度范围
#                 velocities[i] = np.clip(velocities[i], min_bound, max_bound)

#                 # 更新粒子的位置
#                 particles[i] += velocities[i]

#                 # 限制位置范围
#                 particles[i] = np.clip(particles[i], min_bound, max_bound)

#                 # 更新局部最佳位置和全局最佳位置
#                 if objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < objective_func(personal_best_positions[i],parm[0],parm[1],parm[2],parm[3]):
#                     personal_best_positions[i] = particles[i]

#                 if objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3]):
#                     global_best_position = particles[i]
#         return global_best_position, objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3])
#     if mm==-1:
#         # 初始化粒子的局部最佳位置和全局最佳位置
#         personal_best_positions = particles.copy()
#         global_best_position = particles[np.argmin(-objective_func(particles,parm[0],parm[1],parm[2],parm[3]))]

#         # 迭代更新粒子的速度和位置
#         for _ in range(max_iterations):
#             for i in range(num_particles):
#                 # 更新粒子的速度
#                 velocities[i] = (inertia * velocities[i] +
#                                 cognitive_weight * np.random.rand() * (personal_best_positions[i] - particles[i]) +
#                                 social_weight * np.random.rand() * (global_best_position - particles[i]))

#                 # 限制速度范围
#                 velocities[i] = np.clip(velocities[i], min_bound, max_bound)

#                 # 更新粒子的位置
#                 particles[i] += velocities[i]

#                 # 限制位置范围
#                 particles[i] = np.clip(particles[i], min_bound, max_bound)

#                 # 更新局部最佳位置和全局最佳位置
#                 if -objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < -objective_func(personal_best_positions[i],parm[0],parm[1],parm[2],parm[3]):
#                     personal_best_positions[i] = particles[i]

#                 if -objective_func(particles[i],parm[0],parm[1],parm[2],parm[3]) < -objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3]):
#                     global_best_position = particles[i]
#         # 输出结果
#         return global_best_position, -objective_func(global_best_position,parm[0],parm[1],parm[2],parm[3])

# 求解方程 f(x) = y
def solve_equation(f,y):
    equation = lambda x: f(x) - y
    x_initial_guess = 0  # 初始猜测值
    x_solution = fsolve(equation, x_initial_guess)
    return x_solution

        
        

Set={}
youSet={}
resSet={}
resTime={}
for index, row in selected_df.iterrows():
    if index >= 100:
        break
    # 去除NaN值
    data = row.values[~np.isnan(row.values)]
    # 将数据分成x和y坐标对
    x = data[::2] / (3600 )
    y = data[1::2]/1000
    you=[]
    model=[]
    model_select=[]
    you_index={}
    try:
        a1 = np.polyfit(x, y, 1)#线性
        you1 = goodness_of_fit([liner_func(x[p],a1[0],a1[1]) for p in range(len(x))],y)
        you.append(you1)
        print(1)
        model.append({'回归类型':'线性回归','回归系数':a1})
        you_index['线性回归']=you1
        model_select.append((liner_func,a1))
    except:
        pass  
    try:
        a2 = np.polyfit(x, y, 2)#二次
        you2 = goodness_of_fit([erchi_func(x[p],a2[0],a2[1],a2[2]) for p in range(len(x))],y)
        you.append(you2)
        print(2)
        model.append({'回归类型':'二次函数回归','回归系数':a2})
        you_index['二次函数回归']=you2
        print(a2+[0])
        model_select.append((erchi_func,a2))
    except:
        pass
    
    try:
        a3 = np.polyfit(x, y, 3)#三次
        you3 = goodness_of_fit([sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3]) for p in range(len(x))],y)
        you.append(you3)
        print(3)
        model.append({'回归类型':'三次函数回归','回归系数':a3})
        you_index['三次函数回归']=you3
        model_select.append((sanchi_func,a3))
    except:
        pass
    
    #拟合三角函数模型
    try:
        a4,_=optimize.curve_fit(trig_func,x,y)
        you4 = goodness_of_fit([trig_func(x[p],a4[0],a4[1],a4[2]) for p in range(len(x))],y)
        you.append(you4)
        print(4)
        model.append({'回归类型':'三角函数回归','回归系数':a4})
        you_index['三角函数回归']=you4
        model_select.append((trig_func,a4))
        
    except:
        pass
    
    #拟合指数函数模型
    try:
        a5,_=optimize.curve_fit(target_func,x,y)
        print(5)
        you5 = goodness_of_fit([target_func(x[p], a5[0], a5[1], a5[2]) for p in range(len(x))],y)
        you.append(you5)
        
        model.append({'回归类型':'指数函数回归','回归系数':a5})
        you_index['指数函数回归']=you5
        model_select.append((target_func,a5))
    except:
        pass
    
    #拟合对数函数模型
    try:
        a6,_=optimize.curve_fit(hyp_func,x,y)
        print(6)
        you6 = goodness_of_fit([hyp_func(x[p], a6[0],a6[1]) for p in range(len(x))],y)
        you.append(you6)
        model.append({'回归类型':'对数函数回归','回归系数':a6})
        you_index['对数函数回归']=you6
        model_select.append((hyp_func,a6))
    except:
        pass
    
    # model=[f'线性回归，系数为{a1}',f'二次函数回归，系数为{a2}',f'三次函数回归，系数为{a3}',f'三角函数回归，系数为{a4}',f'指数函数回归，系数为{a5}',f'对数函数回归，系数为{a6}']
    res,model_=selected_func(you,model,model_select)
    
    Set[index]=res#选择函数模型，例如：1用三角函数
    youSet[index]=you_index#拟合优度
    # best_position1, best_value1 = particle_swarm_optimization(1,model_[0], 50, 500,model_[1])
    # best_value1=round(float(best_value1),4)
    # print("最小值:",best_value1,'最小位置:',best_position1)
    # best_position2, best_value2 = particle_swarm_optimization(-1,model_[0], 50, 500,model_[1])

    # best_value2=round(float(-best_value2),4)
    # print("最大值:", -best_value2,'最大位置:',best_position2)
    # resSet[index+1]=(y[0],float(-best_value2),x[0],best_position2)
    x1 = symbols('x')
    def y1(x):
        return model_[0](x,*model_[1])
    # if float(best_value1)<0:
    #     best_value1=0.1
    
    # 在第一次检测的的右边的非复数解
    
    solution11 = solve_equation(y1,float(y[0])+6)
    solution1=[]
    for u in solution11:
        if str(u)[-1]!='I' and u>=x[0]:
            solution1.append(u)
       
    solution22 = solve_equation(y1,float(y[0])*1.33)
    solution2=[]
    for u in solution22:
        if str(u)[-1]!='I' and u>=x[0]:
            solution2.append(u)

    if solution1 == [] and solution2 == []:
        resTime[index+1]=-999
    elif solution1 == [] and solution2 != []:
        if solution2[0] <= 48:
            resTime[index+1]=round(solution2[0],4)
        else:
            resTime[index+1]=-999
    elif solution2 == [] and solution1 != []:
        if solution1[0] <= 48:
            resTime[index+1]=round(solution1[0],4)
        else:
            resTime[index+1]=-999
    else:
        if solution1[0] > 48 and solution2[0] > 48:
            resTime[index+1]=-999
        else:
            resTime[index+1]=round(min(solution1[0],solution2[0]),4)
            
    

dicts1 = [{key: value for key, value in Set[row].items()} for row in Set]
df1 = pd.DataFrame(dicts1)
df1.to_excel('血肿1a_100人拟合函数选择结果.xlsx')
dicts2 = [{key: value for key, value in youSet[row].items()} for row in youSet]
df2 = pd.DataFrame(dicts2)
df2.to_excel('血肿1a_100人拟合优度.xlsx')

# kuozan_index={}
# for i in resSet:
#     kuozan_index[i]=[0,1][sgn(resSet[i][3]-resSet[i][2])*(resSet[i][1]-resSet[i][0])>=6 or resSet[i][1]>=1.33*resSet[i][0]]
# df111=pd.DataFrame(columns=['ID','是否扩张'])
# df111['ID']=kuozan_index.keys()
# df111['是否扩张']=kuozan_index.values()
# df111.to_excel('是否扩张.xlsx',index=False)


Time=pd.DataFrame(columns=['ID','扩张时间'])
Time['ID']=resTime.keys()
Time['扩张时间']=resTime.values()
Time.to_excel('扩张时间.xlsx',index=False)

1
2
[-1.89585370e-04  1.42113289e-02  7.22205055e+01]
3
4
5
6
1
2
[ 5.76556821e-05 -1.11144568e-01  5.20770506e+01]
3
4
5
6
1
2
[-7.19239070e-02  3.44037499e+00  7.98029456e+01]
3
4
6
1
2
[ 1.29968130e-03 -4.63365556e-01  4.65634872e+01]
3
4
5
6
1
2
[-4.68017903e-03  5.96324425e-01  1.19673824e+01]
3
4
5
6
1
2
[ 1.90579262e-03 -1.16583388e+00  1.97483631e+02]
3
4
5
6
1
2
[ 2.53605815e-05 -1.03140330e-01  3.40568212e+01]
3
4
5
6
1
2
[-1.01319105e-03  1.80479446e-01  2.93535058e+01]
3
4
5
6
1
2
[-7.50003658e-03  6.93198222e-01  4.08799240e+01]
3
4
5
6
1
2
[ 1.19428638e-04 -1.45196366e-01  3.01109328e+01]
3
4
5
6
1
2
[-1.00978247e-04  2.02329438e-02  4.43986803e+00]
3
4
5
6
1
2
[ 3.06412160e-05 -9.16985383e-02  6.67490766e+01]
3
4
5
6
1
2
[ 1.69072287e-05 -3.35410196e-02  1.49336778e+01]
3
4
5
6
1
2
[ 3.01657560e-04 -1.28465906e-01  3.53631642e+01]
3
4
5
6
1
2
[-1.22302661e-05 -5.42483009e-02  3.80911267e+01]
3
4
5
6
1
2
[ 4.32695055e-03 -5.26995360e-02  5.10763726e+01]
3
4
5
6
1
2
[-1.25